# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    
    #  Implement
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    return pad_sequences(x, padding='post', maxlen=length)
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [6]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    print (preprocess_y.shape)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

(137861, 21)
Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [7]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [8]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    print (input_shape)
    print (output_sequence_length)
    print (english_vocab_size)
    print (french_vocab_size)
    
    model = Sequential()
    model.add(LSTM(units=33, input_shape=input_shape[1:], return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    
    print (model.summary())
    
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)

simple_rnn_model.fit(tmp_x,
                     preproc_french_sentences, 
                     batch_size=1024, 
                     epochs=10, 
                     validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 21, 33)            4620      
_________________________________________________________________
dense_1 (Dense)              (None, 21, 344)           11696     
Total params: 16,316
Trainable params: 16,316
Non-trainable params: 0
_________________________________________________________________
None
(137861, 21, 1)
21
200
345
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 21, 33)            4620      
_________________________________________________________________
dense_2 (Dense)              (None, 21, 345)           11730     
Total params: 16,350
Trainable params: 16,350
Non-trainable params: 0
_________________________________________________________________
None
Trai

## Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [9]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    """
    References:
        * http://www.orbifold.net/default/2017/01/10/embedding-and-tokenizer-in-keras/
    """
    print (input_shape)
    print (output_sequence_length)
    print (english_vocab_size)
    print (french_vocab_size)
    
    # Build the layers
    model = Sequential()
    model.add(Embedding(input_shape=(input_shape[1],), 
                        input_dim=english_vocab_size, 
                        output_dim=50, 
                        input_length=output_sequence_length))

    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    
    print(model.summary())
    
    return model
tests.test_embed_model(embed_model)

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

print (tmp_x[0])

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) +1 )

embed_rnn_model.fit(tmp_x, 
                    preproc_french_sentences, 
                    batch_size=1024, 
                    epochs=10, 
                    validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 50)            9950      
_________________________________________________________________
lstm_3 (LSTM)                (None, 21, 50)            20200     
_________________________________________________________________
dense_3 (Dense)              (None, 21, 344)           17544     
Total params: 47,694
Trainable params: 47,694
Non-trainable params: 0
_________________________________________________________________
None
[17 23  1  8 67  4 39  7  3  1 55  2 44  0  0  0  0  0  0  0  0]
(137861, 21)
21
200
345
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 50)            10000     
_________________________________________________________________
lstm_4 (LSTM)       

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [10]:
from keras.layers import Bidirectional

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    """
    References: 
        * F. Chollet - https://github.com/fchollet/keras/blob/master/examples/imdb_bidirectional_lstm.py
        * https://github.com/farizrahman4u/seq2seq/issues/96
    """ 
    print (input_shape)
    print (output_sequence_length)
    print (english_vocab_size)
    print (french_vocab_size)
    
    # Build the layers
    model = Sequential()
    model.add(Bidirectional(LSTM(units=33, return_sequences=True), input_shape=input_shape[1:]))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    
    print(model.summary())    
    
    return model
    
tests.test_bd_model(bd_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)

bd_rnn_model.fit(tmp_x, 
                 preproc_french_sentences, 
                 batch_size=1024, 
                 epochs=10, 
                 validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 21, 66)            9240      
_________________________________________________________________
dense_5 (Dense)              (None, 21, 344)           23048     
Total params: 32,288
Trainable params: 32,288
Non-trainable params: 0
_________________________________________________________________
None
(137861, 21, 1)
21
200
345
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 21, 66)            9240      
_________________________________________________________________
dense_6 (Dense)              (None, 21, 345)           23115     
Total params: 32,355
Trainable params: 32,355
Non-trainable params: 0
_________________________________________________________________
None
Trai

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [11]:
from keras.layers import RepeatVector

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    """
    REFERENCES:
        * https://github.com/fchollet/keras/issues/395
    """
    
    # Build the layers
    print (input_shape)
    print (output_sequence_length)
    print (english_vocab_size)
    print (french_vocab_size)
    
    model = Sequential()

    ##Encoder
    model.add(Bidirectional(LSTM(units=64), input_shape=input_shape[1:]))
    model.add(Dense(64, activation='softmax'))
    model.add(RepeatVector(output_sequence_length))
    ##Decoder
    model.add(LSTM(units=french_vocab_size, return_sequences=True))    
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    
    print(model.summary())    
    
    return model
    
tests.test_encdec_model(encdec_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_english_sentences.shape[-1])
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-1], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)

encdec_rnn_model.fit(tmp_x, 
                 preproc_french_sentences, 
                 batch_size=1024, 
                 epochs=10, 
                 validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 15, 1)
21
199
344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 128)               33792     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 64)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 21, 344)           562784    
Total params: 604,832
Trainable params: 604,832
Non-trainable params: 0
_________________________________________________________________
None
(137861, 15, 1)
21
200
345
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 128)               33792     
___________

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [12]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras.layers import RepeatVector

from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding


def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    """
    REFERENCES:
        * https://github.com/fchollet/keras/issues/7133 - input_length parameter use to pad input to output size
    """
    
    # Build the layers
    magic_lstm_factor = 1.3
    
    model = Sequential()

    model.add(Embedding(input_dim=english_vocab_size, 
                        output_dim=int(english_vocab_size / 4), 
                        input_shape=(input_shape[1],)))    
    
    model.add(Bidirectional(LSTM(units=int(english_vocab_size * magic_lstm_factor))))
    model.add(Dense(int(french_vocab_size/2), activation='softmax'))
    model.add(RepeatVector(output_sequence_length))
    ##Decoder
    model.add(Bidirectional(LSTM(units=int(french_vocab_size * magic_lstm_factor), return_sequences=True)))    
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])

    
    print(model.input_shape)    
    print(model.summary())    
    
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

(None, 15)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15, 49)            9751      
_________________________________________________________________
bidirectional_5 (Bidirection (None, 516)               635712    
_________________________________________________________________
dense_9 (Dense)              (None, 172)               88924     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 172)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 21, 894)           2217120   
_________________________________________________________________
dense_10 (Dense)             (None, 21, 344)           307880    
Total params: 3,259,387
Trainable params: 3,259,387
Non-trainable params: 0
_______________________________________________________

## Prediction (IMPLEMENTATION)

In [13]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
        
     # Reshaping the input
    tmp_x = x.reshape((-1, x.shape[-1]))
        
    # Train the neural network
    model = model_final(
        tmp_x.shape,
        y.shape[1],
        len(x_tk.word_index) + 1,
        len(y_tk.word_index) + 1)

    model.fit(tmp_x, 
         y, 
         batch_size=1024, 
         epochs=200, 
         validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(None, 15)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 15, 50)            10000     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 520)               646880    
_________________________________________________________________
dense_11 (Dense)             (None, 172)               89612     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 172)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 21, 896)           2225664   
_________________________________________________________________
dense_12 (Dense)             (None, 21, 345)           309465    
Total params: 3,281,621
Trainable params: 3,281,621
Non-trainable params: 0
_______________________________________________________

110288/110288 [==============================] - 68s - loss: 0.2438 - acc: 0.9224 - val_loss: 0.2656 - val_acc: 0.9149
Epoch 55/200
110288/110288 [==============================] - 68s - loss: 0.2343 - acc: 0.9253 - val_loss: 0.2505 - val_acc: 0.9197
Epoch 56/200
110288/110288 [==============================] - 68s - loss: 0.2315 - acc: 0.9256 - val_loss: 0.2438 - val_acc: 0.9222
Epoch 57/200
110288/110288 [==============================] - 68s - loss: 0.2306 - acc: 0.9259 - val_loss: 0.2827 - val_acc: 0.9098
Epoch 58/200
110288/110288 [==============================] - 69s - loss: 0.2208 - acc: 0.9294 - val_loss: 0.2374 - val_acc: 0.9224
Epoch 59/200
110288/110288 [==============================] - 69s - loss: 0.2112 - acc: 0.9327 - val_loss: 0.2261 - val_acc: 0.9274
Epoch 60/200
110288/110288 [==============================] - 69s - loss: 0.2120 - acc: 0.9320 - val_loss: 0.2269 - val_acc: 0.9283
Epoch 61/200
110288/110288 [==============================] - 68s - loss: 0.2044 - acc: 0

110288/110288 [==============================] - 68s - loss: 0.0689 - acc: 0.9808 - val_loss: 0.1004 - val_acc: 0.9731
Epoch 117/200
110288/110288 [==============================] - 68s - loss: 0.0655 - acc: 0.9817 - val_loss: 0.1223 - val_acc: 0.9663
Epoch 118/200
110288/110288 [==============================] - 68s - loss: 0.0695 - acc: 0.9804 - val_loss: 0.1248 - val_acc: 0.9650
Epoch 119/200
110288/110288 [==============================] - 68s - loss: 0.0639 - acc: 0.9818 - val_loss: 0.0974 - val_acc: 0.9737
Epoch 120/200
110288/110288 [==============================] - 68s - loss: 0.0513 - acc: 0.9862 - val_loss: 0.0989 - val_acc: 0.9735
Epoch 121/200
110288/110288 [==============================] - 68s - loss: 0.0506 - acc: 0.9862 - val_loss: 0.0948 - val_acc: 0.9747
Epoch 122/200
110288/110288 [==============================] - 68s - loss: 0.0601 - acc: 0.9833 - val_loss: 0.1046 - val_acc: 0.9712
Epoch 123/200
110288/110288 [==============================] - 68s - loss: 0.0536 -

110288/110288 [==============================] - 68s - loss: 0.0239 - acc: 0.9950 - val_loss: 0.0912 - val_acc: 0.9808
Epoch 178/200
110288/110288 [==============================] - 69s - loss: 0.0227 - acc: 0.9953 - val_loss: 0.0962 - val_acc: 0.9795
Epoch 179/200
110288/110288 [==============================] - 69s - loss: 0.0764 - acc: 0.9821 - val_loss: 0.1856 - val_acc: 0.9531
Epoch 180/200
110288/110288 [==============================] - 69s - loss: 0.0477 - acc: 0.9874 - val_loss: 0.0945 - val_acc: 0.9785
Epoch 181/200
110288/110288 [==============================] - 69s - loss: 0.0223 - acc: 0.9957 - val_loss: 0.0915 - val_acc: 0.9811
Epoch 182/200
110288/110288 [==============================] - 69s - loss: 0.0206 - acc: 0.9962 - val_loss: 0.0900 - val_acc: 0.9807
Epoch 183/200
110288/110288 [==============================] - 69s - loss: 0.0215 - acc: 0.9960 - val_loss: 0.0970 - val_acc: 0.9786
Epoch 184/200
110288/110288 [==============================] - 69s - loss: 0.0310 -

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.